## Prediction Pipeline

In [1]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [3]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [5]:
with open('static/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [6]:
with open('static/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [7]:
vocab = pd.read_csv('static/vocabulary.txt', header=None)
tokens = vocab[0].tolist()

In [8]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

### Preprocessing Function

In [9]:
def preprocessing(text):
    data = pd.DataFrame([text], columns=['tweet'])
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["tweet"] = data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data["tweet"] = data["tweet"].apply(remove_punctuations)
    data["tweet"] = data['tweet'].str.replace('\d+', '', regex=True)
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data["tweet"]

### Vectorizing Function

In [10]:
def vectorizer(ds, vocabulary):
    vectorized_lst = []
    
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
                
        vectorized_lst.append(sentence_lst)
        
    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)
    
    return vectorized_lst_new

In [11]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)
    if prediction == -1:
        return 'Negative'
    else:
        return 'Positive'

In [14]:
def get_sentiment():
    n=0
    while n<5:
        txt= input("Enter Your Text :")
        preprocessed_txt = preprocessing(txt)
        vectorized_txt = vectorizer(preprocessed_txt, tokens)
        prediction = get_prediction(vectorized_txt)
        print('Your Input has ',prediction,' Sentiment')
        n+=1

In [ ]:
get_sentiment()

Enter Your Text : Don't go thieving, it takes money and time to finish everything, there are some liars and money grabbers here.


Your Input has  negative  Sentiment


Enter Your Text : Tried to call several times they are just cutting the call worst service


Your Input has  negative  Sentiment


Enter Your Text : Would give 0 stars if I can.


Your Input has  positive  Sentiment


Enter Your Text : Bad service. Don't recommend here.


Your Input has  negative  Sentiment
